In [1]:
import numpy as np    # we're going to use numpy to process input and output data
import onnxruntime    # to inference ONNX models, we use the ONNX Runtime
import onnx
from onnx import numpy_helper
import urllib.request
import json
import time

# display images in notebook
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

%matplotlib inline

In [2]:
onnx_model_url = "https://huggingface.co/onnxmodelzoo/resnet50_Opset18_torch_hub/resolve/main/resnet50_Opset18_torch_hub.onnx"
imagenet_labels_url = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"

# retrieve our model from the ONNX model zoo
urllib.request.urlretrieve(onnx_model_url, filename="model.onnx")
urllib.request.urlretrieve(imagenet_labels_url, filename="imagenet-simple-labels.json")

('imagenet-simple-labels.json', <http.client.HTTPMessage at 0x71783a754b90>)

In [4]:
import pathlib

project_root = pathlib.Path("..")
ep_lib_path = project_root / "build" / "libiree_onnx_ep.so"

if not ep_lib_path.exists():
    print(f"ERROR: EP library not found at {ep_lib_path}")
print(f"ep library path: {ep_lib_path}")

# Register the EP plugin
onnxruntime.register_execution_provider_library("IREE", str(ep_lib_path))
print("EP plugin registered successfully")

# Get IREE device
ep_devices = onnxruntime.get_ep_devices()
iree_device = None
for dev in ep_devices:
    if dev.ep_name == "IREE":
        iree_device = dev
        break

if not iree_device:
    print("ERROR: IREE EP not found in EP devices")

print(f"IREE EP available (vendor={iree_device.ep_vendor})")


ep library path: ../build/libiree_onnx_ep.so
EP plugin registered successfully
IREE EP available (vendor=IREE)


In [7]:
sess_options = onnxruntime.SessionOptions()
sess_options.log_severity_level = 1
provider_options = {
    "device": "hip://0",
    "target_arch": "gfx1201",
    "opt_level": "O3",
}
sess_options.add_provider_for_devices([iree_device], provider_options)

# Run the model on the backend
session = onnxruntime.InferenceSession('model.onnx', sess_options=sess_options)

*************** EP Error ***************
EP Error /onnxruntime_src/onnxruntime/core/session/plugin_ep/ep_plugin_provider_interfaces.cc:55 virtual std::unique_ptr<onnxruntime::IExecutionProvider> onnxruntime::PluginExecutionProviderFactory::CreateProvider(const OrtSessionOptions&, const OrtLogger&) Error creating execution provider: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : IREE EP: 'device' option must be specified
 when using None
Falling back to ['CPUExecutionProvider'] and retrying.
****************************************


2026-01-13 14:46:23.228375573 [I:onnxruntime:, inference_session.cc:606 TraceSessionOptions] Session Options {  execution_mode:0 execution_order:DEFAULT enable_profiling:0 optimized_model_filepath:"" enable_mem_pattern:1 enable_mem_reuse:1 enable_cpu_mem_arena:1 profile_file_prefix:onnxruntime_profile_ session_logid: session_log_severity_level:1 session_log_verbosity_level:0 max_num_graph_transformation_steps:10 graph_optimization_level:4 intra_op_param:OrtThreadPoolParams { thread_pool_size: 0 auto_set_affinity: 0 allow_spinning: 1 dynamic_block_base_: 0 stack_size: 0 affinity_str:  set_denormal_as_zero: 0 } inter_op_param:OrtThreadPoolParams { thread_pool_size: 0 auto_set_affinity: 0 allow_spinning: 1 dynamic_block_base_: 0 stack_size: 0 affinity_str:  set_denormal_as_zero: 0 } use_per_session_threads:1 thread_pool_allow_spinning:1 use_deterministic_compute:0 ep_selection_policy:0 config_options: {   ep.iree.device: hip://0  ep.iree.target_arch: gfx1201  ep.iree.opt_level: O3 } }
202

In [ ]:
def load_labels(path):
    with open(path) as f:
        data = json.load(f)
    return np.asarray(data)

def preprocess(input_data):
    # convert the input data into the float32 input
    img_data = input_data.astype('float32')

    #normalize
    mean_vec = np.array([0.485, 0.456, 0.406])
    stddev_vec = np.array([0.229, 0.224, 0.225])
    norm_img_data = np.zeros(img_data.shape).astype('float32')
    for i in range(img_data.shape[0]):
        norm_img_data[i,:,:] = (img_data[i,:,:]/255 - mean_vec[i]) / stddev_vec[i]
        
    #add batch channel
    norm_img_data = norm_img_data.reshape(1, 3, 224, 224).astype('float32')
    return norm_img_data

def prepare_image(image):
    return preprocess(np.array(image).transpose(2, 0, 1))

def softmax(x):
    x = x.reshape(-1)
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def postprocess(result):
    return softmax(np.array(result)).tolist()

In [ ]:
labels = load_labels('imagenet-simple-labels.json')
image0 = Image.open('images/dog.jpg')
image1 = Image.open('images/plane.jpg')

_, axarr = plt.subplots(2, 1)
axarr[0].axis("off")
axarr[0].imshow(image0)
axarr[1].axis("off")
axarr[1].imshow(image1)

input0 = prepare_image(image0)
input1 = prepare_image(image1)

In [ ]:
def run_inference(inp, image):
    start = time.time()
    raw_result = session.run([], {"x": inp})
    end = time.time()
    res = postprocess(raw_result)

    inference_time = np.round((end - start) * 1000, 2)
    
    print('========================================')
    print('Inference time: ' + str(inference_time) + " ms")
    print('========================================')

    idx = np.argmax(res)

    print('========================================')
    print('Final top prediction is: ' + labels[idx])
    print('========================================')

    sort_idx = np.flip(np.squeeze(np.argsort(res)))
    print('============ Top 5 labels are: ============================')
    print(labels[sort_idx[:5]])
    print('===========================================================')
    
    plt.axis('off')
    display_image = plt.imshow(image)

In [ ]:
run_inference(input0, image0)

In [ ]:
run_inference(input1, image1)